In [ ]:
# To generate the skincare routine for the user

In [3]:
pip install langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_openai import OpenAI

In [5]:
import chromadb


In [6]:
import os
from Password import openai_key  # Importing openai_key from Password.py
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = openai_key  # Using the imported openai_key

# Initialize embeddings and Chroma vector database
embeddings = OpenAIEmbeddings()
vectordb = Chroma(embedding_function=embeddings, collection_name="Amanda", persist_directory="chroma_persist")

# Add texts to Chroma
vectors = vectordb.add_texts(["I have sensitive skin", "I like l'Oreal products", "Never recommend me sunscreen"])

# Define a custom prompt template for skincare routines
prompt_template = """
Hi, I am your personal AI assistant from AINA! How may I assist you today?

A skincare routine has the following 7 steps:

1. Cleanser
2. Exfoliator
3. Toner
4. Serum
5. Eye cream
6. Moisturizer
7. Sunscreen

Recommend actual skincare products based on the user's preferences, considering their name, age, gender, and skin tone.

Always output in JSON in this format: {{"step #": "product name"}}

Context: Name: {name}, Age: {age}, Gender: {gender}, Skin Tone: {skin_tone}

Question: What is the best skincare routine for {name}, a {age}-year-old {gender} with {skin_tone} skin?

Please answer just like a human, and ask any additional queries to fine-tune your response for a more personalized answer.
"""

# Prompt Templates
skincare_routine_prompt = PromptTemplate(
    input_variables=['name', 'age', 'gender', 'skin_tone'],
    template=prompt_template
)

# Memory management setup
skincare_memory = ConversationBufferMemory(input_key='skin_tone', memory_key='routine_history')

# Define and initialize LLM and Chains
llm = OpenAI(temperature=0.8)
skincare_chain = LLMChain(
    llm=llm, prompt=skincare_routine_prompt, verbose=True, output_key='skincare_routine', memory=skincare_memory)

parent_chain = SequentialChain(
    chains=[skincare_chain], input_variables=['name', 'age', 'gender', 'skin_tone'], output_variables=['skincare_routine'], verbose=True)

# Input handling updated for new variables
input_details = {
    "name": input("Enter your name: ").strip(),
    "age": input("Enter your age: ").strip(),
    "gender": input("Enter your gender: ").strip(),
    "skin_tone": input("Enter your skin tone type: ").strip()
}

if all(input_details.values()):  # Ensuring all inputs are provided
    result = parent_chain(input_details)
    print(f"Skincare Routine for {input_details['name']}:")
    print(result['skincare_routine'])

    # Optionally, store the conversation in vectordb for future retrieval
    vectordb.add_texts([result['skincare_routine']])
else:
    print("Please ensure all details (name, age, gender, skin tone) are entered.")


Enter your name:  Amanda
Enter your age:  34
Enter your gender:  female
Enter your skin tone type:  oily




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Hi, I am your personal AI assistant from AINA! How may I assist you today?

A skincare routine has the following 7 steps:

1. Cleanser
2. Exfoliator
3. Toner
4. Serum
5. Eye cream
6. Moisturizer
7. Sunscreen

Recommend actual skincare products based on the user's preferences, considering their name, age, gender, and skin tone.

Always output in JSON in this format: {"step #": "product name"}

Context: Name: Amanda, Age: 34, Gender: female, Skin Tone: oily

Question: What is the best skincare routine for Amanda, a 34-year-old female with oily skin?

Please answer just like a human, and ask any additional queries to fine-tune your response for a more personalized answer.


> Finished chain.

> Finished chain.
Skincare Routine for Amanda:

Based on Amanda's age, gender, and skin type, here is a recommended skincare routine:

1. Cleanser: "Garnier SkinActive Micellar Foaming Face Wash" 
2